<a href="https://colab.research.google.com/github/edwardleetenafly/LA-net/blob/main/lanet_week1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install nibabel scikit-image


Mounted at /content/drive


In [ ]:
import torch
print("CUDA available:", torch.cuda.is_available())


CUDA available: True


In [ ]:
!ls "/content/drive"


MyDrive


In [ ]:
!ls "/content/drive/MyDrive"


'12 Kinds of People BEFORE AN EXAM.gslides'
'12 Kinds of People DURING AN EXAM.gslides'
'12 Sentence Summary.gdoc'
 2021-04-08_07.04.33.png
'3rd Star Power Ideas.gslides'
'5th Grade Literary Choice Board.gdoc'
'6.1 answers.gdoc'
'6th Grade Practice Test.gdoc'
'6th Grade Practice Test.pdf'
'9 TITAN POWERS - ATTACK ON TITAN.gdoc'
 ADADDE~1.gdoc
 ADADDE~1.pdf
'ADMIN CARD.gslides'
 algebra1_equations_two_word-1.gdoc
 algebra1_equations_two_word-1.pdf
'All 52 Star Powers Tier List.gslides'
"All Characters for Warrior's Cove.gslides"
'ALL CLASH ROYALE.gdoc'
"All Gamemode Ideas for Warrior's Cove.gslides"
'All Lightsaber Forms.gslides'
'ALL MCU MOVIES RANKED.gslides'
'Allosaurus vs Ceratosaurus.gslides'
'ALL STAR WARS MOVIES RANKED.gslides'
'All Vocabs: .gdoc'
'Answers to Solving Multi-Step Equations.gdoc'
'ATEEZ All Songs Line Distributions.gsheet'
'Attorney Review Letter - signed.gdoc'
'Attorney Review Letter - signed.pdf'
"AUDIENCE'S VS TOMATAOERS.gdoc"
'AURORA LINE DISTRIBUTION.gsheet'
'A

In [ ]:
import csv
from pathlib import Path

root = Path("/content/drive/MyDrive/la_net")
images = root / "imagesTr"
labels = root / "labelsTr"

manifest = root / "manifest.csv"

rows = []

# IMPORTANT: look for .nii instead of .nii.gz
for img_path in sorted(images.glob("*.nii")):
    case_id = img_path.stem  # e.g., "la_007"

    # ALSO look for .nii in labels
    label_path = labels / f"{case_id}.nii"

    if not label_path.exists():
        print("Skipping missing label for", case_id)
        continue

    rows.append({
        "case_id": case_id,
        "image_path": str(img_path),
        "mask_path": str(label_path),
    })

with manifest.open("w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=["case_id","image_path","mask_path"])
    writer.writeheader()
    writer.writerows(rows)

print("Wrote manifest.csv with", len(rows), "cases")


Wrote manifest.csv with 20 cases


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

root = Path("/content/drive/MyDrive/la_net")
manifest_path = root / "manifest.csv"

df = pd.read_csv(manifest_path)
case_ids = df["case_id"].values

rng = np.random.default_rng(1337)
rng.shuffle(case_ids)

n = len(case_ids)
train = case_ids[:int(0.70*n)]
val   = case_ids[int(0.70*n):int(0.85*n)]
test  = case_ids[int(0.85*n):]

splits = (
    [{"case_id": cid, "split": "train"} for cid in train] +
    [{"case_id": cid, "split": "val"} for cid in val] +
    [{"case_id": cid, "split": "test"} for cid in test]
)

splits_df = pd.DataFrame(splits)
splits_df.to_csv(root / "splits.csv", index=False)

splits_df["split"].value_counts()


,count
split,
train,14
val,3
test,3


In [ ]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from skimage import exposure
from pathlib import Path
import pandas as pd

# Your dataset root
root = Path("/content/drive/MyDrive/la_net")

# Load manifest.csv
df = pd.read_csv(root / "manifest.csv")

# Directory to save previews
out_dir = root / "clahe_previews"
out_dir.mkdir(exist_ok=True)

def load_middle_slice(path):
    """Load the 3D volume and return the middle axial slice."""
    vol = nib.load(path).get_fdata()
    mid_z = vol.shape[2] // 2
    slice_ = vol[:, :, mid_z]
    return slice_

# Choose first 3 cases for preview
sample_cases = df["case_id"].values[:3]

# CLAHE parameters
clip_limits = [1.5, 2.0, 2.5]
tile_grids  = [6, 8, 10]

for cid in sample_cases:
    row = df[df["case_id"] == cid].iloc[0]
    img_path = row["image_path"]

    raw = load_middle_slice(img_path)

    for clip in clip_limits:
        for grid in tile_grids:

            # Apply CLAHE
            raw_norm = (raw - np.min(raw)) / (np.ptp(raw) + 1e-8)
            clahe = exposure.equalize_adapthist(
                raw_norm, clip_limit=clip, kernel_size=grid
            )

            # Plot and save
            plt.figure(figsize=(7, 3))

            plt.subplot(1, 2, 1)
            plt.imshow(raw, cmap="gray")
            plt.title("Raw")
            plt.axis("off")

            plt.subplot(1, 2, 2)
            plt.imshow(clahe, cmap="gray")
            plt.title(f"CLAHE clip={clip}, grid={grid}")
            plt.axis("off")

            fname = f"{cid}_clip-{clip}_grid-{grid}.png"
            plt.savefig(out_dir / fname, dpi=150, bbox_inches="tight")
            plt.close()

            print("Saved:", fname)

print("DONE ✔ All CLAHE preview images generated.")


Saved: la_003_clip-1.5_grid-6.png
Saved: la_003_clip-1.5_grid-8.png
Saved: la_003_clip-1.5_grid-10.png
Saved: la_003_clip-2.0_grid-6.png
Saved: la_003_clip-2.0_grid-8.png
Saved: la_003_clip-2.0_grid-10.png
Saved: la_003_clip-2.5_grid-6.png
Saved: la_003_clip-2.5_grid-8.png
Saved: la_003_clip-2.5_grid-10.png
Saved: la_004_clip-1.5_grid-6.png
Saved: la_004_clip-1.5_grid-8.png
Saved: la_004_clip-1.5_grid-10.png
Saved: la_004_clip-2.0_grid-6.png
Saved: la_004_clip-2.0_grid-8.png
Saved: la_004_clip-2.0_grid-10.png
Saved: la_004_clip-2.5_grid-6.png
Saved: la_004_clip-2.5_grid-8.png
Saved: la_004_clip-2.5_grid-10.png
Saved: la_005_clip-1.5_grid-6.png
Saved: la_005_clip-1.5_grid-8.png
Saved: la_005_clip-1.5_grid-10.png
Saved: la_005_clip-2.0_grid-6.png
Saved: la_005_clip-2.0_grid-8.png
Saved: la_005_clip-2.0_grid-10.png
Saved: la_005_clip-2.5_grid-6.png
Saved: la_005_clip-2.5_grid-8.png
Saved: la_005_clip-2.5_grid-10.png
DONE ✔ All CLAHE preview images generated.
